# Contrato de Integración Flight On Time

Versio

## Variables Entrada API

El siguiente `JSON` corresponde a la entrada de la API

```
{
    "aerolinea": "AZ",
    "origen": "GIG",
    "destino": "GRU",
    "fecha_partida": "2025-11-10T14:30:00",
    "distancia_km": 350
}
```

| Nombre | Tipo | Ejemplo | Descripción | Observacíón | Pre procesamiento | 
|-----------|-----------|-----------|-----------|-----------|-----------|
| `aerolinea` | Cadena de texto | `DL` | Código IATA Aerolinea | Códigos disponibles en `aerolineas.csv` | No |
| `origen` | Cadena de texto | `ATL` | Código IATA, único por aeropuerto | Códigos disponibles en `ciudades.csv` | No |
| `destino` | Cadena de texto | `CLE` | Código IATA, único por aeropuerto | Códigos disponibles en `ciudades.csv` | No |
| `fecha_partida` | Cadena de texto | `2025-11-10T14:30:00` |Formato AAAA-MM-DDTHH:MM:SS, Hora 0-24| T es un separador | Si |
| `distancia_km` | Númerico entero | 750 | Distancia entre aeropuertos en kilometros | <span style="color:red">Pendiente listar por ruta para validación</span>. | Si |

### Validaciones sugeridas

- Códigos Aerolinea y ciudades válidos según archivos CSV.
- Formato hora fecha válido.
- Tipo de dato

### Preprocesamiento variables entrada API

### `fecha_partida`

A partir de esta fecha partida se obtienen 3 variables de entrada al modelo `HORA_PARTIDA`,`DIA_SEMANA`,`ES_HORA_PICO`:

In [20]:
from datetime import datetime

# Cadena original
fecha_partida = "2025-12-11T15:30:00"

# Convertir a datetime
fecha_dt = datetime.strptime(fecha_partida, "%Y-%m-%dT%H:%M:%S")

print(f"Tipo: {type(fecha_dt)}")
print(f"Año: {fecha_dt.year}")
print(f"Mes: {fecha_dt.month}")
print(f"Día: {fecha_dt.day}")
print(f"Hora: {fecha_dt.hour}")
print(f"Minuto: {fecha_dt.minute}")
print(f"Segundo: {fecha_dt.second}")


Tipo: <class 'datetime.datetime'>
Año: 2025
Mes: 12
Día: 11
Hora: 15
Minuto: 30
Segundo: 0


In [21]:
HORA_PARTIDA = fecha_dt.hour 
DIA_SEMANA = fecha_dt.weekday()  # Lunes=0, Domingo=6
ES_HORA_PICO = 1 if fecha_dt.hour >= 15 else 0  # Hora pico >= 15:00

print(f"\nHORA_PARTIDA: {HORA_PARTIDA}")
print(f"DIA_SEMANA: {DIA_SEMANA}")  # 0=Lunes, 1=Martes, etc.
print(f"ES_HORA_PICO: {ES_HORA_PICO}")



HORA_PARTIDA: 15
DIA_SEMANA: 3
ES_HORA_PICO: 1


In [22]:
from datetime import datetime

def preproc_fecha(fecha_partida):
    fecha_dt = datetime.strptime(fecha_partida, "%Y-%m-%dT%H:%M:%S")
    HORA_PARTIDA = fecha_dt.hour 
    DIA_SEMANA = fecha_dt.weekday()  # Lunes=0, Domingo=6
    ES_HORA_PICO = 1 if fecha_dt.hour >= 15 else 0  # Hora pico >= 15:00
    return HORA_PARTIDA, DIA_SEMANA, ES_HORA_PICO
    

In [23]:
print(preproc_fecha("2025-12-11T15:30:00"))
print(preproc_fecha("2021-09-25T09:30:00"))
print(preproc_fecha("2019-11-02T18:30:00"))
print(preproc_fecha("2023-01-01T00:00:00"))
print(preproc_fecha("2009-09-08T16:25:00"))

(15, 3, 1)
(9, 5, 0)
(18, 5, 1)
(0, 6, 0)
(16, 1, 1)


### `distancia_km`

Se utiliza el factor de conversión de 1 km = 0.621371 millas y se convierte al entero mas cercano

In [24]:
def km_a_millas(km):
    FACTOR = 0.621371
    return int(FACTOR*km)

In [25]:
print(f"1. 750 km son {km_a_millas(750)} millas")
print(f"2. 0 km son {km_a_millas(0)} millas")
print(f"3. 1 km son {km_a_millas(1)} millas")
print(f"4. 10 km son {km_a_millas(10)} millas")
print(f"5. 42.195 km son {km_a_millas(42.195)} millas")
print(f"6. 1000 km son {km_a_millas(1000)} millas")
print(f"7. 12.5 km son {km_a_millas(12.5)} millas")

1. 750 km son 466 millas
2. 0 km son 0 millas
3. 1 km son 0 millas
4. 10 km son 6 millas
5. 42.195 km son 26 millas
6. 1000 km son 621 millas
7. 12.5 km son 7 millas


## Variables Salida API

```
{
"prevision": "Retrasado",
"probabilidad": 0.78,
"codigo_prediccion": 1,
"timestamp": "2025-11-10T14:30:00Z"
}
```

| Nombre | Tipo | Ejemplo | Descripción | Observacíón | Pos procesamiento
|-----------|-----------|-----------|-----------|-----------|-----------|
| `prevision` | Cadena de texto | Retrasado | Etiqueta predicha "Retrasado" o "A tiempo" | Depende de codigo_prediccion | Si |
| `probabilidad` | flotante 2 decimales | 0.78 | valor de probabilidad entre 0 y 1 | Directa del modelo | Si |
| `codigo_prediccion` | Númerico entero | 1 | Puede ser 0 o 1 | Directa del modelo | No |
| `timestamp` | Cadena de texto | "2025-11-10T14:30:00Z" | Formato AAAA-MM-DDTHH:MM:SS, Hora 0-24 | Hora y fecha de la predicción | Si |

## Variables entrada al modelo

| Nombre | Tipo | Ejemplo | Descripción | Observacíón | Pre procesamiento
|-----------|-----------|-----------|-----------|-----------|-----------|
| `AIRLINE_CODE` | Cadena de texto | `DL` | Código IATA Aerolinea | Códigos disponibles en `aerolineas.csv` | No |
| `ORIGIN` | Cadena de texto | `ATL` | Código IATA, único por aeropuerto | Códigos disponibles en `ciudades.csv` | No |
| `DEST` | Cadena de texto | `CLE` | Código IATA, único por aeropuerto | Códigos disponibles en `ciudades.csv` | No |
| `DISTANCE` | Númerico entero | 750 | Distancia entre aeropuertos en millas | <span style="color:red">Pendiente obtener equivalencia por ruta, hacer pruebas con valor númerico</span>. | Si |
| `HORA_PARTIDA` | Númerico entero | 17   | Hora aproximada de salida | Variable que se obtiene del campo fecha | Si |
| `DIA_SEMANA` | Númerico entero | 5 | Día de la semana (0-6) inicia 0 -> Domingo | Variable que se obtiene del campo fecha | Si |
| `ES_HORA_PICO` | Númerico entero |1| Binario(0,1), `1 si hora >=15` | Variable que se obtiene del campo fecha | Si |

## Variables salida del modelo

| Nombre | Tipo | Ejemplo | Descripción | Observacíón | Pos procesamiento|
|-----------|-----------|-----------|-----------|-----------|-----------|
| `resultado` | Númerico entero | Retrasado | Etiqueta predicha "Retrasado" o "A tiempo" | Depende de codigo_prediccion | Si |
| `probabilidad` | flotante 2 decimales | 0.78 | valor de probabilidad entre 0 y 1 | Directa del modelo | Si |

In [26]:
import joblib
import pandas as pd

from datetime import datetime, timezone


def obtener_timestamp():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# Cargar modelo
modelo = joblib.load('modelo_flightontime_xgboost_final.pkl')

# Crear datos prueba
datos = [['DL', 'ATL', 'CLE', 750, 17, 5, 1]]

# Crear DataFrame
columnas = ['AIRLINE_CODE', 'ORIGIN', 'DEST', 'DISTANCE', 
            'HORA_PARTIDA', 'DIA_SEMANA', 'ES_HORA_PICO']

df = pd.DataFrame(datos, columns=columnas)

# Predecir
resultado = modelo.predict(df)[0]
probs = modelo.predict_proba(df)[0]

def prob_prediccion(resultado, probs):
    return float(f"{probs[resultado]:.2f}")
    
def etiqueta_pred(resultado):
    if resultado == 1:
        etiqueta = "Retrasado"
    else:
        etiqueta = "A tiempo"
    return etiqueta

print(f"prevision: {etiqueta_pred(resultado)}")
print(f"probabilidad: {prob_prediccion(resultado, probs)}")
print(f"codigo_prediccion: {resultado}")
print(f"timestamp: {obtener_timestamp()}")

prevision: Retrasado
probabilidad: 0.63
codigo_prediccion: 1
timestamp: 2026-01-16T05:27:39Z


In [27]:
import joblib
import pandas as pd
import numpy as np

# Cargar modelo
modelo = joblib.load('modelo_flightontime_xgboost_final.pkl')

# Columnas en orden exacto
columnas = ['AIRLINE_CODE', 'ORIGIN', 'DEST', 'DISTANCE', 
            'HORA_PARTIDA', 'DIA_SEMANA', 'ES_HORA_PICO']

# ===========================================
# VECTORES DE PRUEBA REALISTAS
# ===========================================

vectores_prueba = [
    # 1. Ejemplo original de la tabla
    {
        'nombre': 'Ejemplo tabla (DL ATL->CLE)',
        'datos': ['DL', 'ATL', 'CLE', 750, 17, 5, 1]
    },
    
    # 2. Vuelo corto, mañana, día laboral
    {
        'nombre': 'Vuelo corto mañana (AA ORD->MDW)',
        'datos': ['AA', 'ORD', 'MDW', 15, 9, 1, 0]  # Lunes 9 AM
    },
    
    # 3. Vuelo transcontinental, tarde, viernes
    {
        'nombre': 'Transcontinental viernes (UA SFO->JFK)',
        'datos': ['UA', 'SFO', 'JFK', 2576, 16, 4, 1]  # Viernes 4 PM
    },
    
    # 4. Vuelo internacional simulado
    {
        'nombre': 'Vuelo internacional (DL ATL->MEX)',
        'datos': ['DL', 'ATL', 'MEX', 1112, 13, 3, 0]  # Miércoles 1 PM
    },
    
    # 5. Vuelo nocturno
    {
        'nombre': 'Vuelo rojo (WN LAS->DEN)',
        'datos': ['WN', 'LAS', 'DEN', 628, 22, 6, 0]  # Sábado 10 PM
    },
    
    # 6. Vuelo hora pico extremo
    {
        'nombre': 'Hora pico (AA DFW->ORD)',
        'datos': ['AA', 'DFW', 'ORD', 802, 17, 2, 1]  # Martes 5 PM
    },
    
    # 7. Vuelo domingo temprano
    {
        'nombre': 'Domingo temprano (DL JFK->LAX)',
        'datos': ['DL', 'JFK', 'LAX', 2475, 7, 0, 0]  # Domingo 7 AM
    },
    
    # 8. Vuelo distancia media
    {
        'nombre': 'Distancia media (UA DEN->SEA)',
        'datos': ['UA', 'DEN', 'SEA', 1024, 11, 3, 0]  # Miércoles 11 AM
    },
    
    # 9. Vuelo fin de semana tarde
    {
        'nombre': 'Fin de semana tarde (AS SEA->LAX)',
        'datos': ['AS', 'SEA', 'LAX', 954, 14, 5, 0]  # Sábado 2 PM
    },
    
    # 10. Vuelo muy corto
    {
        'nombre': 'Vuelo muy corto (WN HOU->DAL)',
        'datos': ['WN', 'HOU', 'DAL', 239, 10, 4, 0]  # Viernes 10 AM
    }
]

print(" PRUEBAS DEL MODELO DE RETRASO DE VUELOS")
print("="*60)

for i, prueba in enumerate(vectores_prueba, 1):
    # Crear DataFrame
    datos = [prueba['datos']]
    df = pd.DataFrame(datos, columns=columnas)
    
    # Realizar predicción
    prediccion = modelo.predict(df)[0]
    
    # Mostrar resultados
    print(f"\n{i}. {prueba['nombre']}")
    print(f"   Datos: {prueba['datos']}")
    print(f"   Predicción: {prediccion} ({'RETRASADO' if prediccion == 1 else 'A TIEMPO'})")
    
    # Si el modelo tiene predict_proba, mostrar probabilidades
    if hasattr(modelo, 'predict_proba'):
        try:
            proba = modelo.predict_proba(df)[0]
            # Asumimos que la primera clase es 0 (a tiempo) y la segunda es 1 (retrasado)
            if len(proba) >= 2:
                print(f"   Probabilidad retraso: {proba[1]:.2%}")
                print(f"   Probabilidad a tiempo: {proba[0]:.2%}")
        except Exception as e:
            print(f"   Error obteniendo probabilidades: {e}")

 PRUEBAS DEL MODELO DE RETRASO DE VUELOS

1. Ejemplo tabla (DL ATL->CLE)
   Datos: ['DL', 'ATL', 'CLE', 750, 17, 5, 1]
   Predicción: 1 (RETRASADO)
   Probabilidad retraso: 63.18%
   Probabilidad a tiempo: 36.82%

2. Vuelo corto mañana (AA ORD->MDW)
   Datos: ['AA', 'ORD', 'MDW', 15, 9, 1, 0]
   Predicción: 0 (A TIEMPO)
   Probabilidad retraso: 18.31%
   Probabilidad a tiempo: 81.69%

3. Transcontinental viernes (UA SFO->JFK)
   Datos: ['UA', 'SFO', 'JFK', 2576, 16, 4, 1]
   Predicción: 0 (A TIEMPO)
   Probabilidad retraso: 32.92%
   Probabilidad a tiempo: 67.08%

4. Vuelo internacional (DL ATL->MEX)
   Datos: ['DL', 'ATL', 'MEX', 1112, 13, 3, 0]
   Predicción: 1 (RETRASADO)
   Probabilidad retraso: 52.42%
   Probabilidad a tiempo: 47.58%

5. Vuelo rojo (WN LAS->DEN)
   Datos: ['WN', 'LAS', 'DEN', 628, 22, 6, 0]
   Predicción: 1 (RETRASADO)
   Probabilidad retraso: 70.61%
   Probabilidad a tiempo: 29.39%

6. Hora pico (AA DFW->ORD)
   Datos: ['AA', 'DFW', 'ORD', 802, 17, 2, 1]
   Predi

In [28]:
import joblib
import pandas as pd

def probar_probabilidades(modelo_path='modelo_flightontime_xgboost_final.pkl'):
    """
    Prueba si el modelo puede dar probabilidades y cómo interpretarlas
    """
    # Cargar modelo
    modelo = joblib.load(modelo_path)
    
    print(" PRUEBA DE PROBABILIDADES DEL MODELO")
    print("="*50)
    
    # 1. Verificar capacidades
    print(f"\n Capacidades del modelo:")
    print(f"   Tiene predict_proba: {hasattr(modelo, 'predict_proba')}")
    
    if hasattr(modelo, 'classes_'):
        print(f"   Clases: {modelo.classes_}")
        print(f"   Número de clases: {len(modelo.classes_)}")
    
    # 2. Crear varios casos de prueba
    casos_prueba = [
        ('Caso 1: Retraso probable', ['DL', 'ATL', 'LGA', 760, 18, 4, 1]),
        ('Caso 2: A tiempo probable', ['AA', 'DFW', 'AUS', 182, 10, 2, 0]),
        ('Caso 3: Caso intermedio', ['UA', 'ORD', 'DEN', 888, 15, 3, 1])
    ]
    
    columnas = ['AIRLINE_CODE', 'ORIGIN', 'DEST', 'DISTANCE', 
                'HORA_PARTIDA', 'DIA_SEMANA', 'ES_HORA_PICO']
    
    print("\n Resultados por caso:")
    
    for nombre, datos in casos_prueba:
        df = pd.DataFrame([datos], columns=columnas)
        
        # Predicción simple
        pred = modelo.predict(df)[0]
        
        print(f"\n{nombre}:")
        print(f"  Datos: {datos}")
        print(f"  Predicción: {pred} = {'RETRASADO' if pred == 1 else 'A TIEMPO'}")
        


        proba = modelo.predict_proba(df)[0]
        print(f"  Probabilidades: {proba}")
        
        # Interpretar según número de clases
        if len(proba) == 2:
            print(f"  P(A tiempo): {proba[0]:.2%}")
            print(f"  P(Retrasado): {proba[1]:.2%}")
            
            # Calcular confianza
            confianza = max(proba)
            clase_confiable = 'A tiempo' if proba[0] > proba[1] else 'Retrasado'
            print(f"  Confianza: {confianza:.2%} ({clase_confiable})")
            
        elif len(proba) == 1:
            print(f"  Probabilidad única: {proba[0]:.2%}")
            print("   Modelo parece ser de regresión, no clasificación")
                    

    
    # 3. Prueba con múltiples instancias a la vez
    print("\n" + "="*50)
    print(" PRUEBA CON MÚLTIPLES INSTANCIAS:")
    
    multiples_datos = [
        ['DL', 'ATL', 'CLE', 750, 17, 5, 1],
        ['AA', 'ORD', 'DFW', 802, 9, 1, 0],
        ['UA', 'LAX', 'EWR', 2454, 20, 6, 0]
    ]
    
    df_multi = pd.DataFrame(multiples_datos, columns=columnas)
    
    # Predicciones múltiples
    predicciones = modelo.predict(df_multi)
    print(f"\nPredicciones múltiples: {predicciones}")
    
    if hasattr(modelo, 'predict_proba'):
        try:
            probabilidades = modelo.predict_proba(df_multi)
            print(f"\nProbabilidades múltiples:")
            for i, proba in enumerate(probabilidades):
                if len(proba) == 2:
                    print(f"  Vuelo {i+1}: P(Retraso) = {proba[1]:.2%}")
        except:
            pass


probar_probabilidades()

 PRUEBA DE PROBABILIDADES DEL MODELO

 Capacidades del modelo:
   Tiene predict_proba: True
   Clases: [0 1]
   Número de clases: 2

 Resultados por caso:

Caso 1: Retraso probable:
  Datos: ['DL', 'ATL', 'LGA', 760, 18, 4, 1]
  Predicción: 1 = RETRASADO
  Probabilidades: [0.22138035 0.77861965]
  P(A tiempo): 22.14%
  P(Retrasado): 77.86%
  Confianza: 77.86% (Retrasado)

Caso 2: A tiempo probable:
  Datos: ['AA', 'DFW', 'AUS', 182, 10, 2, 0]
  Predicción: 0 = A TIEMPO
  Probabilidades: [0.7218011 0.2781989]
  P(A tiempo): 72.18%
  P(Retrasado): 27.82%
  Confianza: 72.18% (A tiempo)

Caso 3: Caso intermedio:
  Datos: ['UA', 'ORD', 'DEN', 888, 15, 3, 1]
  Predicción: 1 = RETRASADO
  Probabilidades: [0.4274611 0.5725389]
  P(A tiempo): 42.75%
  P(Retrasado): 57.25%
  Confianza: 57.25% (Retrasado)

 PRUEBA CON MÚLTIPLES INSTANCIAS:

Predicciones múltiples: [1 1 0]

Probabilidades múltiples:
  Vuelo 1: P(Retraso) = 63.18%
  Vuelo 2: P(Retraso) = 50.48%
  Vuelo 3: P(Retraso) = 49.07%
